# Sweep Guide

## Init

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
from pprint import pprint
import numpy as np
import qcodes as qc

from labcore.measurement import *

## Intro

The way we would like to measure looks something like this:

```python
    >>> m = Measurement(
    ...     sweep_object,
    ...     setup=setup_function,
    ...     finish=teardown_function,
    ... )
    ... m.run(**config_options)
```

The reason why we would like to define the measurement in an object is to be able to have access to main characteristics (such as the type of data acquired during the run) in advance, before any measurement code is executed. This will also be useful for serializing measurements.

The most tricky aspect here is to define what the ``sweep`` object is. Often we define the flow in an experiment by directly using for-loops and the like. But that will not allow us the desired goals. We need to come up with an object-based description that makes the essence of the flow clear without executing it.

### Main requirements for sweeps

- a sweep object can be iterated over. each step performs some actions, and returns some data.
  that means, we want to be able to use it as follows.  We call the dictionary that is produced for each step a **record**.
  
    ```python
      >>> for data in sweep_object:
      ...    print(data)
      {variable_1: some_data, variable_2: more_data}
      {variable_1: different_data, variable_2: more_different_data}
    ```
 
- a sweep object is defined by a pointer and 0 or more actions. 
    - A pointer is again an iterable, and the coordinates it traverses may or may not be known in advance. 
    - Actions are callables that may take the values the pointer is returning at each iteration as arguments. Each action is executed once for each pointer.
  
  A simple sweep can then be defined like this:
    
    ```python
        >>> sweep_object = Sweep(range(5), func_1, func_2)
        
    ```
  
  Executing this sweep will loop over the range, and will then call ``func_1(i)`` and subsequently ``func_2(i)``, for each ``i`` in the range from 0 to 4.
  
  
- Sweeps can be combined in ways that result in nesting, zipping, or appending. Combining sweeps again results in a sweep. This will allow us to construct modular measurements from pre-defined blocks.

- After definition, but before execution, we can easily infer what records the sweep produces.

## Examples

In the following first look a prototypical measurement protocol.
After that we have a closer look at individual components.

### Jumping right in

#### A prototypical, slightly non-trivial measurement protocol

Lets say we have the following task:

1. our basic measurement is to record the phase as a function of frequency (for instance with a VNA)
2. we do this as function of some other parameter that tunes something in the experiment (maybe a voltage)
3. finally, we need to average out some slow noise, so we repeat everything some number of times
4. the tuning might heat up the sample, so we want to keep an eye on it. We want to measure the fridge temperature once per repetition.

Being good with organizing our tools and prefering a modular way of putting together measurements, we have already defined the main building blocks so we can re-use them. They might even be located in some module that import for a particular class of experiments. 

The functions that return data we want to save later are decorated with ``recording``, to describe what kind of data they produce. (Note: the decorator is not necessary; we can also use the function ``record_as``, as we'll see below)

In [3]:
# these are just some dummy functions that follow the recipe outlined above.

def tune_something(param):
    print(f'adjusting something in-situ using param={param}')

@recording(
    independent('frq', type='array', unit='Hz'), 
    dependent('phase', depends_on=['frq'], type='array', unit='rad')
)
def measure_frequency_trace(npoints=3):
    return np.arange(1,npoints+1)*1e6, np.random.normal(size=npoints)


@recording(dependent('temperature', unit='K'))
def measure_temperature():
    return np.random.normal(loc=0.1, scale=0.01)

We now use these functions to assemble the measurement loop. It works like this:

- The matrix multiplication operator ``@`` creates a nested loop of sweeps (the outer loop are the repetitions, the inner loop the tuning of the parameter).
- The multiplication operator ``*`` is like an inner product (or zip), i.e., results in element-wise combination. That is, for each value of ``param`` we run ``measure_frequency_trace``.
- The addition operator ``+`` appends sweeps or actions to sweeps. I.e., ``measure_temperature`` is executed once after each ``param`` sweep.

Finally, we can, even without running any measurement code, already look at the data that the loop will produce.

In [4]:
sweep = (
    sweep_parameter('repetition', range(3))
    @ (sweep_parameter('param', range(3), tune_something)
       * measure_frequency_trace
       + measure_temperature)
)

pprint(sweep.get_data_specs())
print()

(repetition, param, frq, phase(repetition, param, frq), temperature(repetition))



Running the sweep is very easy -- we can simply iterate over it. Each sweep coordinate produces a data set that by default contains everything that has been annotated as recording data (the ``sweep_parameter`` function implicitly generates a record for the parameter that is being varied).

In [5]:
for data in sweep:
    print(data)

adjusting something in-situ using param=0
{'repetition': 0, 'param': 0, 'frq': array([1000000., 2000000., 3000000.]), 'phase': array([-1.10076157,  2.84741127, -0.31184165]), 'temperature': None}
adjusting something in-situ using param=1
{'repetition': 0, 'param': 1, 'frq': array([1000000., 2000000., 3000000.]), 'phase': array([-0.40743812,  0.30309396,  0.06591226]), 'temperature': None}
adjusting something in-situ using param=2
{'repetition': 0, 'param': 2, 'frq': array([1000000., 2000000., 3000000.]), 'phase': array([-0.8933407 ,  0.37264877, -1.3794646 ]), 'temperature': None}
{'repetition': 0, 'param': None, 'frq': None, 'phase': None, 'temperature': 0.10825390503990606}
adjusting something in-situ using param=0
{'repetition': 1, 'param': 0, 'frq': array([1000000., 2000000., 3000000.]), 'phase': array([ 1.09678375, -0.12856661, -0.59978759]), 'temperature': None}
adjusting something in-situ using param=1
{'repetition': 1, 'param': 1, 'frq': array([1000000., 2000000., 3000000.]), '

#### A QCoDeS parameter sweep

Sweeps are often done over qcodes parameters, and the data we acquire may also come from parameters.

In this minimal example we set a parameter (``x``) to a range of values, and get data from another parameter for each set value.

In [6]:
from qcodes import Parameter

def measure_stuff():
    return np.random.normal()

x = Parameter('x', set_cmd=lambda x: print(f'setting x to {x}'), initial_value=0)
data = Parameter('data', get_cmd=lambda: np.random.normal())

for record in sweep_parameter(x, range(3), get_parameter(data)):
    print(record)

setting x to 0
setting x to 0
{'x': 0, 'data': -1.8709973583239499}
setting x to 1
{'x': 1, 'data': -0.9951114987590062}
setting x to 2
{'x': 2, 'data': 1.5057536595964158}


### Constructing single sweeps

The main ingredients for a single sweep is 1) a pointer iterable, and 2) a variable number of actions to execute at each iteration step.
Both pointer and actions may generate records.

The most bare example would look something like this:

In [7]:
for data in Sweep(range(3)):
    print(data)

{}
{}
{}


We loop over the iterable, but only empty records are generated.

We can use ``record_as`` to indicate that generated values should be recorded:

In [8]:
def my_func():
    return 0

sweep = Sweep(
    record_as(range(3), independent('x')), # this is the pointer. We specify 'x' as an independent (we control it)
    record_as(my_func, 'y') # y is not declared as independent; 
                            # dependent (on what it depends is partially determined by the sweep) is the default.
)

pprint(sweep.get_data_specs())

for data in sweep:
    print(data)

(x, y(x))
{'x': 0, 'y': 0}
{'x': 1, 'y': 0}
{'x': 2, 'y': 0}


A more convenient way of doing exactly the same thing:

In [9]:
sweep = sweep_parameter('x', range(3), record_as(my_func, 'y'))

for data in sweep:
    print(data)

{'x': 0, 'y': 0}
{'x': 1, 'y': 0}
{'x': 2, 'y': 0}


Elements can also produce records with multiple parameters:

In [10]:
def my_func():
    return 1, 2

sweep = Sweep(
    record_as(zip(range(3), ['a', 'b', 'c']), independent('number'), independent('string')), # a pointer with two parameters
    record_as(my_func, 'one', 'two')
)

pprint(sweep.get_data_specs())

for data in sweep:
    print(data)

(number, string, one(number, string), two(number, string))
{'number': 0, 'string': 'a', 'one': 1, 'two': 2}
{'number': 1, 'string': 'b', 'one': 1, 'two': 2}
{'number': 2, 'string': 'c', 'one': 1, 'two': 2}


### Specifying options before executing a sweep

Many functions we are using take optional parameters we only want to specify just before executing the sweep (but are constant throughout the sweep). 

If we don't want to resort to global variables we can do so by using ``Sweep.set_action_opts``. 
It accepts the names of action functions as keywords, and dictionaries containing keyword arguments to pass to those functions as value.
Keywords specified in this way always override key words that are passed around internally in the sweep (See further below for an explanation on how parameters are passed internally). 

In [11]:
def test_fun(value, a_property=0):
    print('inside test_fun:')
    print(f"value: {value}, property: {a_property}")
    print()
    return value

def another_fun(another_value, *args, **kwargs):
    print('inside another_fun:')
    print(f"my value: {another_value}")
    print(f"other stuff:", args, kwargs)
    print()

sweep_1 = sweep_parameter('value', range(3), record_as(test_fun, dependent('data')))
sweep_2 = sweep_parameter('another_value', range(2), another_fun)

sweep = (
    sweep_1
    @ sweep_2
)

sweep.set_action_opts(
    test_fun = dict(a_property=1),
    another_fun = dict(another_value='Hello', another_property=True)
)

for data in sweep:
    print("Data:", data)
    print()

inside test_fun:
value: 0, property: 1

inside another_fun:
my value: Hello
other stuff: () {'value': 0, 'data': 0, 'another_property': True}

Data: {'value': 0, 'data': 0, 'another_value': 0}

inside another_fun:
my value: Hello
other stuff: () {'value': 0, 'data': 0, 'another_property': True}

Data: {'value': 0, 'data': 0, 'another_value': 1}

inside test_fun:
value: 1, property: 1

inside another_fun:
my value: Hello
other stuff: () {'value': 1, 'data': 1, 'another_property': True}

Data: {'value': 1, 'data': 1, 'another_value': 0}

inside another_fun:
my value: Hello
other stuff: () {'value': 1, 'data': 1, 'another_property': True}

Data: {'value': 1, 'data': 1, 'another_value': 1}

inside test_fun:
value: 2, property: 1

inside another_fun:
my value: Hello
other stuff: () {'value': 2, 'data': 2, 'another_property': True}

Data: {'value': 2, 'data': 2, 'another_value': 0}

inside another_fun:
my value: Hello
other stuff: () {'value': 2, 'data': 2, 'another_property': True}

Data: {

### Appending

A simple example for how to append sweeps to each other. 
The `sweep_parameter` function creates a 1D sweep over a parameter that we can specify just with a name. It will automatically result in a returned record with that name when it is set.

We create two sweeps that consist of a parameter sweep and an action that returns a random number as dependent variable. 
The sweep parameter will automatically be inserted as a dependency in that case.
Here, the annotation of the return data is done using ``record_as``. This allows us to use the same function twice with different return data names.

We finally attach another function call to the sweep. It is executed only once at the very end (internally it's made into a 'null sweep'). It is also not annotated at all, so won't record any data.

Note: ``Sweep.return_none`` controls whether we include data fields that have returned nothing during setting a pointer or executing an action. It can be set on the class or the instance of a particular sweep.
Setting it to true (the default) guarantees that each data spec of the sweep has an entry per sweep point, even if it is ``None``.

In [12]:
def get_random_number():
    return np.random.rand()

def get_another_random_number():
    print('rolling the dice!')
    return np.random.rand()


# change this to see what happens.
Sweep.record_none = False

sweep_1 = sweep_parameter('x', range(3), record_as(get_random_number, dependent('y')))
sweep_2 = sweep_parameter('a', range(4), record_as(get_random_number, dependent('b')))

for data in sweep_1 + sweep_2 + get_another_random_number:
    print(data)
    
# just to set it back for the other examples
Sweep.record_none = True

{'x': 0, 'y': 0.4094965023520305}
{'x': 1, 'y': 0.37484391951956075}
{'x': 2, 'y': 0.9726320893513312}
{'a': 0, 'b': 0.6166582281823296}
{'a': 1, 'b': 0.5551469169493615}
{'a': 2, 'b': 0.6494322774816023}
{'a': 3, 'b': 0.6739660392861476}
rolling the dice!
{}


### Multiplying

By multiplying we refer to an inner product, i.e., the result is basically what you'd expect from ``zip``-ing two iterables.

Simplest case: we have a sweep and want to attach another action to each sweep point:

In [13]:
sweep = (
    sweep_parameter('x', range(3), record_as(get_random_number, dependent('data_1')))
    * record_as(get_random_number, dependent('data_2'))
)

pprint(sweep.get_data_specs())
print()

for data in sweep:
    print(data)

(x, data_1(x), data_2(x))

{'x': 0, 'data_1': 0.7616293736584214, 'data_2': 0.7799067412792655}
{'x': 1, 'data_1': 0.205254849101963, 'data_2': 0.6265510518114651}
{'x': 2, 'data_1': 0.8664803370425319, 'data_2': 0.29921238592233745}


If the two objects we want to combine are both sweeps, then we get zip-like behavior (and dependencies stay separate).

In [14]:
sweep = (
    sweep_parameter('x', range(3), record_as(get_random_number, dependent('data_1')))
    * sweep_parameter('y', range(5), record_as(get_random_number, dependent('data_2')))
)

pprint(sweep.get_data_specs())
print()

for data in sweep:
    print(data)

(x, data_1(x), y, data_2(y))

{'x': 0, 'data_1': 0.008267876269323349, 'y': 0, 'data_2': 0.8693576160091558}
{'x': 1, 'data_1': 0.308242305613855, 'y': 1, 'data_2': 0.4334851201197726}
{'x': 2, 'data_1': 0.29961367507351766, 'y': 2, 'data_2': 0.2125085208556461}


### Nesting sweeps

The most simple example:

Sweep parameters (say, 'x', 'y', and 'z') against each other, and perform a measurement at each point.
For syntactic brevity we're using the matrix multiplication ('@') operator for nesting (you can also use ``labcore.measurement.sweep.nest_sweeps``).

In [15]:
@recording(dependent('my_data'))
def measure_something():
    return np.random.rand()

my_sweep = (
    sweep_parameter('x', range(3)) 
    @ sweep_parameter('y', np.linspace(0,1,3))
    @ sweep_parameter('z', range(-5, -2))
    @ measure_something
)

for data in my_sweep:
    print(data)

{'x': 0, 'y': 0.0, 'z': -5, 'my_data': 0.5463647593417098}
{'x': 0, 'y': 0.0, 'z': -4, 'my_data': 0.42039580778126084}
{'x': 0, 'y': 0.0, 'z': -3, 'my_data': 0.0863297392560125}
{'x': 0, 'y': 0.5, 'z': -5, 'my_data': 0.6667065949897614}
{'x': 0, 'y': 0.5, 'z': -4, 'my_data': 0.5640608701912103}
{'x': 0, 'y': 0.5, 'z': -3, 'my_data': 0.5423047888881569}
{'x': 0, 'y': 1.0, 'z': -5, 'my_data': 0.937230710906636}
{'x': 0, 'y': 1.0, 'z': -4, 'my_data': 0.7702985778019462}
{'x': 0, 'y': 1.0, 'z': -3, 'my_data': 0.22377862008935256}
{'x': 1, 'y': 0.0, 'z': -5, 'my_data': 0.1985299090073276}
{'x': 1, 'y': 0.0, 'z': -4, 'my_data': 0.6573053083491838}
{'x': 1, 'y': 0.0, 'z': -3, 'my_data': 0.22783041062601284}
{'x': 1, 'y': 0.5, 'z': -5, 'my_data': 0.5941299018923976}
{'x': 1, 'y': 0.5, 'z': -4, 'my_data': 0.7251513449604327}
{'x': 1, 'y': 0.5, 'z': -3, 'my_data': 0.26055331694114103}
{'x': 1, 'y': 1.0, 'z': -5, 'my_data': 0.25674732058929683}
{'x': 1, 'y': 1.0, 'z': -4, 'my_data': 0.43059626650

The outer loops can be more complex sweeps as well -- for example, we can also execute measurements on each nesting level.

In [16]:
def measure_something():
    return np.random.rand()

sweep_1 = sweep_parameter('x', range(3), record_as(measure_something, 'a'))
sweep_2 = sweep_parameter('y', range(4), record_as(measure_something, 'b'))

for data in sweep_1 @ sweep_2 @ record_as(get_random_number, 'more_data'):
    print(data)

{'x': 0, 'a': 0.5873625661621503, 'y': 0, 'b': 0.574052863282486, 'more_data': 0.5442260125411729}
{'x': 0, 'a': 0.5873625661621503, 'y': 1, 'b': 0.617226345202642, 'more_data': 0.8957756985108019}
{'x': 0, 'a': 0.5873625661621503, 'y': 2, 'b': 0.8514859887878549, 'more_data': 0.04578646455435931}
{'x': 0, 'a': 0.5873625661621503, 'y': 3, 'b': 0.9586527378963369, 'more_data': 0.8447622676501859}
{'x': 1, 'a': 0.2267792537892548, 'y': 0, 'b': 0.1905793083792563, 'more_data': 0.7864745760222425}
{'x': 1, 'a': 0.2267792537892548, 'y': 1, 'b': 0.6854490064042752, 'more_data': 0.2897639351841491}
{'x': 1, 'a': 0.2267792537892548, 'y': 2, 'b': 0.6403558109006632, 'more_data': 0.8858329848843944}
{'x': 1, 'a': 0.2267792537892548, 'y': 3, 'b': 0.7128289569479528, 'more_data': 0.03623318741786408}
{'x': 2, 'a': 0.31909103654188875, 'y': 0, 'b': 0.2896579960615011, 'more_data': 0.1413170850245894}
{'x': 2, 'a': 0.31909103654188875, 'y': 1, 'b': 0.504754413563884, 'more_data': 0.11278947130653594

### Combining operators and more complex constructs

A not uncommon task in quantum circuits:

1. sweep some parameter. We're interested in some response of the system on this parameter.
2. Hovewer, changing this parameter changes an operation point of our sample. To deal with this we need to perform an auxiliary measurement, and based on its result we set some other parameter. (A familiar case: we increase a drive power that is expected to have some effect on a qubit. But after changing the power we need to first find the readout resonator frequency because of the changed Stark shift.)
3. After this 'calibration step' we can now measure the response we're actually looking for.

Further, we want to record not only the data that is taken in the last step, but also the calibration data.

All together this leads us to a measurement structure as below. As before, the building blocks are modular and relatively self-contained, and might be part of some package.

The data this will produce is two records per sweep parameter (``x``, creatively): one that contains the calibration measurement and result, and one for the actual measurement.

In [17]:
@recording(
    independent('readout_probe_points', type='array'),
    dependent('readout_probe_response', type='array', depends_on=['readout_probe_points']),
    dependent('readout_point')  # note: we don't make this dependent on readout_probe_points
)
def calibrate_readout():
    """An example for a readout calibration."""
    pts = np.linspace(0, 1, 3)
    response = np.random.normal(size=3, scale=10)
    result = response.max()  # some analysis of the calibration measurement
    return pts, response, result
    

def set_operation_point(value):
    print(f'setting the readout point to {value}')
    

# the actual measurement function accepts the argument `readout_point` which will be passed from the calibration.
@recording(
    independent('probe_frequency', type='array'),
    dependent('probe_response', type='array', depends_on=['probe_frequency']),
)
def perform_measurement(readout_point):
    set_operation_point(readout_point)
    return np.arange(3)+10, np.random.normal(size=3, loc=readout_point)

# because of the way brackets are set here, we need to make a sweep out of the calibrate_readout 
# function. this is easily done with the 'once' function, which creates a length-1 sweep with no additional 
# return data.
sweep = (
    sweep_parameter('x', range(3))
    @ (once(calibrate_readout) + perform_measurement)
)

for data in sweep:
    pprint(data)

{'probe_frequency': None,
 'probe_response': None,
 'readout_point': 11.284998339002213,
 'readout_probe_points': array([0. , 0.5, 1. ]),
 'readout_probe_response': array([ 1.12849983e+01,  1.37628561e-02, -1.65627537e+01]),
 'x': 0}
setting the readout point to 11.284998339002213
{'probe_frequency': array([10, 11, 12]),
 'probe_response': array([12.02333575, 10.35975474, 12.11792401]),
 'readout_point': None,
 'readout_probe_points': None,
 'readout_probe_response': None,
 'x': 0}
{'probe_frequency': None,
 'probe_response': None,
 'readout_point': 19.138799706077503,
 'readout_probe_points': array([0. , 0.5, 1. ]),
 'readout_probe_response': array([ 19.13879971,  11.48703869, -10.75048573]),
 'x': 1}
setting the readout point to 19.138799706077503
{'probe_frequency': array([10, 11, 12]),
 'probe_response': array([18.43368321, 19.96660532, 20.0550161 ]),
 'readout_point': None,
 'readout_probe_points': None,
 'readout_probe_response': None,
 'x': 1}
{'probe_frequency': None,
 'probe_r

In [18]:
pprint(sweep.get_data_specs())

(x,
 readout_probe_points,
 readout_probe_response(x, readout_probe_points),
 readout_point(x),
 probe_frequency,
 probe_response(x, probe_frequency))


## Looking a bit deeper

### Annotating objects for recording

These are just some snippets that show how we can annotate records and what the results are.

Without record annotations sweeps still are executed, but no data is recorded:

In [19]:
sweep_object = Sweep(range(3), lambda x: np.random.normal(size=x))
for data in sweep_object:
    print(data)

{}
{}
{}


The underlying object we use for declaring data returns is ``DataSpec``. ``ds`` is just a shortcut that points to its constructor. 

``independent`` (or ``indep``) is creates a DataSpec with ``depends_on=None``, and ``dependent`` creates a DataSpec with ``depends_on=[]``. If ``depends_on`` is ``None``, it will remain independent even when the annotated object is embedded in larger structures. If it is `[]` then more dependencies are added automatically.

In [20]:
# defining some example measurement functions without short-hand notations

@recording(DataSpec('x'), DataSpec('y', depends_on=['x'], type='array'))
def measure_stuff(n, *args, **kwargs):
    return n, np.random.normal(size=n)

@recording(ds('a'))
def set_stuff(x, *args, **kwargs):
    return x

measure_stuff(1), set_stuff(1)

({'x': 1, 'y': array([-0.45867552])}, {'a': 1})

Generators can also be annotated to produce data.

In [21]:
@recording(ds('a'))
def make_sequence(n):
    for i in range(n):
        yield i
        
for data in make_sequence(3):
    print(data)

{'a': 0}
{'a': 1}
{'a': 2}


Using ``record_as`` is a practical way of annotating records just before executing, 
independently of an earlier function definition. 
This works with functions and generators as well.

In [22]:
def get_some_data(n):
    return np.random.normal(size=n)

record_as(get_some_data, ds('random_var'))(3)

{'random_var': array([ 1.71492645, -0.2915872 ,  0.94294788])}

record also provides a simple short hand for labelling regular iterables and iterators:
it automatically removes surplus values, and always returns values for all specified data, 
even if the annotated object does not return in (missing values will be ``None``)

In [23]:
for data in record_as(zip(np.linspace(0,1,6), np.arange(6)), ds('x')):
    print(data)
    
for data in record_as(zip(np.linspace(0,1,6), np.arange(6)), ds('x'), ds('y')):
    print(data)
    
for data in record_as(np.linspace(0,1,6), ds('x'), ds('y')):
    print(data)

{'x': 0.0}
{'x': 0.2}
{'x': 0.4}
{'x': 0.6000000000000001}
{'x': 0.8}
{'x': 1.0}
{'x': 0.0, 'y': 0}
{'x': 0.2, 'y': 1}
{'x': 0.4, 'y': 2}
{'x': 0.6000000000000001, 'y': 3}
{'x': 0.8, 'y': 4}
{'x': 1.0, 'y': 5}
{'x': 0.0, 'y': None}
{'x': 0.2, 'y': None}
{'x': 0.4, 'y': None}
{'x': 0.6000000000000001, 'y': None}
{'x': 0.8, 'y': None}
{'x': 1.0, 'y': None}


### Passing parameters in a sweep

Everything that is generated by functions (or pointers and sweeps) can in principle be passed on to subsequently executed elements. 

When there's no record annotations arguments may still be passed as positional arguments:

In [24]:
def test(*args, **kwargs):
    print(args, kwargs)
    return 100
    
for data in Sweep(range(3), test):
    print(data)

(0,) {}
{}
(1,) {}
{}
(2,) {}
{}


Because it would get too confusing otherwise, positional arguments only get passed originating from a pointer to all actions in a single sweep. In the following example the two ``test`` functions in the first sweep are passed the same integer, whereas the function in the second sweep is only receiving (``x``, ``True``) or (``y``, ``False``). Note that the return of ``test`` is not passed to any other object.

In [25]:
for data in Sweep(range(3), test, test) * Sweep(zip(['x', 'y'], [True, False]), test):
    print(data)

(0,) {}
(0,) {}
('x', True) {}
{}
(1,) {}
(1,) {}
('y', False) {}
{}
(2,) {}
(2,) {}


In the previous example, ``test`` received any arguments passed to it because its signature included variational positional arguments (``*args``). The situation changes when this is not the case. The function is receiving only arguments that it can accept.

In [26]:
def test_2(x=2):
    print(x)
    return True

for data in Sweep(zip([1,2], [3,4]), test_2):
    pass

1
2


We are more flexible when we use keyword arguments. Here, the rule is:

1. All records produced are passed to all subsequent functions in the sweep (even across different sub-sweeps!) that accept the keyword. 
2. If a pointer yields non-annotated values, these are still used as positional arguments, but only where accepted, and with higher priority given to keywords. 
3. using ``lambda`` and ``record_as`` allow pretty simple translation of records and argument names to avoid conflicts.
4. some elementary control over passing behavior is provided by ``Sweep.pass_on_returns`` and ``Sweep.pass_on_none``.
    1. ``Sweep.pass_on_returns`` defaults to ``True``. If set to ``False``, nothing will be passed on.
    2. ``Sweep.pass_on_none`` defaults to ``False``. If set to ``False`` the behavior is such that records that are ``None`` will not be passed on further. (Because ``None`` is typically indicating that function did not return anything as data even though a record was declard using ``recording`` or ``record_as``).
    3. Note: At the moment we can set those only globally using the class attribute; a likely update in the future.

Some examples:

In [27]:
def test(x, y, z=5):
    print("my three arguments:", x, y, z)
    return x, y, z

def print_all_args(*args, **kwargs):
    print("arguments at the end of the line:", args, kwargs)
    
    
for data in sweep_parameter('x', range(3), test):
    print("data:", data)
    
print()
sweep = (
    sweep_parameter('y', range(3), record_as(test, dependent('xx'), dependent('yy'), dependent('zz')))
    @ print_all_args
)
for data in sweep:
    print("data:", data)
    
print()
sweep = (
    sweep_parameter('y', range(3), record_as(test, dependent('xx'), dependent('yy'), dependent('zz')))
    @ record_as(lambda xx, yy, zz: test(xx, yy, zz), dependent('some'), dependent('different'), dependent('names'))
    @ print_all_args
    + print_all_args
)
for data in sweep:
    print("data:", data)

my three arguments: 0 None 5
data: {'x': 0}
my three arguments: 1 None 5
data: {'x': 1}
my three arguments: 2 None 5
data: {'x': 2}

my three arguments: None 0 5
arguments at the end of the line: () {'y': 0, 'yy': 0, 'zz': 5}
data: {'y': 0, 'xx': None, 'yy': 0, 'zz': 5}
my three arguments: None 1 5
arguments at the end of the line: () {'y': 1, 'yy': 1, 'zz': 5}
data: {'y': 1, 'xx': None, 'yy': 1, 'zz': 5}
my three arguments: None 2 5
arguments at the end of the line: () {'y': 2, 'yy': 2, 'zz': 5}
data: {'y': 2, 'xx': None, 'yy': 2, 'zz': 5}

my three arguments: None 0 5
my three arguments: None 0 5
arguments at the end of the line: () {'y': 0, 'yy': 0, 'zz': 5, 'different': 0, 'names': 5}
data: {'y': 0, 'xx': None, 'yy': 0, 'zz': 5, 'some': None, 'different': 0, 'names': 5}
my three arguments: None 1 5
my three arguments: None 1 5
arguments at the end of the line: () {'y': 1, 'yy': 1, 'zz': 5, 'different': 1, 'names': 5}
data: {'y': 1, 'xx': None, 'yy': 1, 'zz': 5, 'some': None, 'diffe

In [28]:
Sweep.pass_on_returns = False

sweep = (
    sweep_parameter('y', range(3), record_as(test, dependent('xx'), dependent('yy'), dependent('zz')))
    @ print_all_args
)
for data in sweep:
    print("data:", data)

# setting back to default
Sweep.pass_on_returns = True

my three arguments: None None 5
arguments at the end of the line: () {}
data: {'y': 0, 'xx': None, 'yy': None, 'zz': 5}
my three arguments: None None 5
arguments at the end of the line: () {}
data: {'y': 1, 'xx': None, 'yy': None, 'zz': 5}
my three arguments: None None 5
arguments at the end of the line: () {}
data: {'y': 2, 'xx': None, 'yy': None, 'zz': 5}


In [29]:
Sweep.pass_on_none = True

sweep = (
    sweep_parameter('y', range(3), record_as(test, dependent('xx'), dependent('yy'), dependent('zz')))
    @ print_all_args
)

for data in sweep:
    print("data:", data)
    
# setting back to default
Sweep.pass_on_none = False

my three arguments: None 0 5
arguments at the end of the line: (None,) {'y': 0, 'xx': None, 'yy': 0, 'zz': 5}
data: {'y': 0, 'xx': None, 'yy': 0, 'zz': 5}
my three arguments: None 1 5
arguments at the end of the line: (None,) {'y': 1, 'xx': None, 'yy': 1, 'zz': 5}
data: {'y': 1, 'xx': None, 'yy': 1, 'zz': 5}
my three arguments: None 2 5
arguments at the end of the line: (None,) {'y': 2, 'xx': None, 'yy': 2, 'zz': 5}
data: {'y': 2, 'xx': None, 'yy': 2, 'zz': 5}
